In [ ]:
# Run this cell to download the course materials and functions.
import os
import sys

# Clone the repository
!git clone https://github.com/Stephen-Robbins/Math_of_Finance.git

# Change directory to the repository's root
%cd Math_of_Finance

# --- Important: Add the 'scripts' directory to Python's search path ---
# Get the absolute path to the 'scripts' folder. This is crucial for portability.
scripts_path = os.path.join(os.getcwd(), "scripts")

# Add the scripts directory to Python's path
if scripts_path not in sys.path:  # Avoid adding it multiple times
    sys.path.insert(0, scripts_path)

# Geometric Brownian Motion (GBM)

We model the stock price $S_t$ as a stochastic process satisfying the following stochastic differential equation (SDE):

$$
dS_t = \mu S_t \, dt + \sigma S_t \, dW_t,
$$

where:

- $\mu$ is the drift coefficient, representing the expected rate of return.
- $\sigma$ is the volatility coefficient, measuring the standard deviation of the stock’s returns.
- $dW_t$ is the increment of a standard Brownian motion (or Wiener process).

Under this model, the logarithm of the stock price is normally distributed, ensuring that \(S_t > 0\) for all \(t\). This SDE underpins many financial models, including the Black-Scholes option pricing formula.

In the plots below, we compare a simulated GBM path with real historical stock price data.


In [ ]:
from plot import plot_gbm_path

# Plot a simulated GBM path
t, S_paths = plot_gbm_path(S0=100, mu=0.07, sigma=0.3, T=1, N=252,  n_paths=1)


In [ ]:
from plot import plot_stock_path
# Plot a real stock path (e.g., Apple Inc. from 2021-01-01 to 2022-01-01)
data = plot_stock_path('AAPL', '2021-01-01', '2022-01-01')

In [ ]:
from plot import plot_log_returns_bell_curve

# Use the function to plot log returns and the fitted normal distribution for a given ticker and date range.
plot_log_returns_bell_curve('V', '2021-01-01', '2024-01-01', bins=100)

# Log-Normal Distribution of Stock Prices Under Geometric Brownian Motion

Assume that the stock price $S_t$ follows a Geometric Brownian Motion (GBM) described by the stochastic differential equation:

$$
dS_t = \mu S_t\, dt + \sigma S_t\, dW_t,
$$

This implies that the stock price at time $T$ is given by:

$$
S_T = S_t \exp\left\{ \left(\mu - \frac{1}{2}\sigma^2\right)(T-t) + \sigma\sqrt{T-t}\, Z \right\},
$$

where $Z \sim N(0,1)$. Consequently, $S_T$ is log-normally distributed. Its probability density function (pdf) is given by:

$$
f_{S_T}(x) = \frac{1}{x\, \sigma_1 \sqrt{2\pi}} \exp\left\{ -\frac{(\ln(x) - \mu_1)^2}{2\sigma_1^2} \right\},
$$

with the parameters:

$$
\mu_1 = \ln[S_t] + \left(\mu - \frac{1}{2}\sigma^2\right)(T-t),
$$

$$
\sigma_1^2 = \sigma^2 (T-t).
$$

Below, we illustrate this with a plot using the parameters:  
- Annual drift: $\mu = 0.07$  
- Initial stock price: $S_0 = 100$  
- Annual volatility: $\sigma = 0.1$  
- Time horizon: $T = 2$ years


In [ ]:
from plot import plot_log_normal_distribution
mu = 0.07       # annual drift
S0 = 100        # initial stock price
sigma = 0.1    # annual volatility
T = 2          # 1 year
plot_log_normal_distribution(mu, S0, sigma, T)

### Black-Scholes Formula for a European Call and Put Option

The price of a European call option $C$ is given by:

$$ c(t) = S(t) \cdot \Phi(d_1) - e^{-r(T-t)} E\cdot \Phi(d_2) $$



where:

- $S(t)$: Stock price at time $t$
- $E$: Strike price of the option
- $T$: Time to expiration
- $r$: Risk-free interest rate
- $\Phi(x)$: Cumulative distribution function of the standard normal distribution
- $d_1$ and $d_2$ are calculated as:

$$ d_1 = \frac{\ln(S(t) / E) + (r + \sigma^2 / 2)(T-t)}{\sigma \sqrt{T-t}} $$
$$ d_2 = d_1 - \sigma \sqrt{T-t} $$

- $\sigma$: Volatility (Standard Deviation) of the stock's returns


In [ ]:
#Example
from finance import black_scholes_call_price
S0 = 100      # Current stock price
E = 100       # Strike price
r = 0.05      # 5% annual risk-free rate
T = 1         # 1 year until expiration
sigma = 0.2   # 20% annual volatility

price = black_scholes_call_price(S0, E, r, T, sigma)
print(f"The European call option price is: {price:.2f}")


### Estimating Annualized Volatility

Given historical stock prices $S(n)$ for $N+1$ days, we can estimate daily returns $R_n$ as:

$$ R_n = \frac{S(n) - S(n - 1)}{S(n - 1)} $$

Then, we compute the daily mean return $\hat{\mu}_d$ and daily volatility $\hat{\sigma}_d$:

$$ \hat{\mu}_d = \frac{1}{N} \sum_{n=1}^{N} \ln(1 + R_n) $$
$$ \hat{\sigma}_d = \sqrt{\frac{1}{N - 1} \sum_{n=1}^{N} (\ln(1 + R_n) - \hat{\mu}_d)^2} $$

To annualize these estimates, assuming 252 trading days in a year, the formulas are:

$$ \hat{\mu} = 252 \hat{\mu}_d $$
$$ \hat{\sigma} = \hat{\sigma}_d \sqrt{252} $$

These estimates provide insights into the average return and the risk or uncertainty of the stock's return, which are pivotal for financial modeling and risk management.

In [ ]:
from finance import annualized_volatility

daily_returns = [0.01, -0.005, 0.003, 0.02, -0.01, 0.005, -0.002]

ann_vol, daily_vol, daily_mu = annualized_volatility(daily_returns)

print(f"Daily Mean Log Return (μ_d): {daily_mu:.5f}")
print(f"Daily Volatility (σ_d): {daily_vol:.5f}")
print(f"Annualized Volatility (σ): {ann_vol:.2%}")

In [ ]:
# Import required functions
from finance import (
    historical_annualized_volatility, 
    get_current_stock_price, 
    get_current_option_price, 
    time_to_maturity, 
    black_scholes_call_price
)

# Stock and date range for historical volatility computation
ticker = 'AAPL'
start_date = '2024-02-05'
end_date = '2025-02-05'

# Option parameters
expiration = "2026-01-16"  
strike = 250.0
option_type = 'call'

# Risk-free rate for Black-Scholes calculation (annualized)
risk_free_rate = 0.0418
annualized_vol, daily_vol, daily_mean = historical_annualized_volatility(ticker, start_date, end_date)
price = get_current_stock_price(ticker)
T = time_to_maturity(expiration)
price_option = black_scholes_call_price(price, strike, risk_free_rate, T, annualized_vol)
opt_price = get_current_option_price(ticker, expiration, strike, option_type) #Real price



print("--------------------------------------------------")
print(f"Ticker: {ticker}")
print("--------------------------------------------------")
print(f"Daily Mean Log Return (μ_d): {daily_mean:.5f}")
print(f"Daily Volatility (σ_d): {daily_vol:.5f}")
print(f"Annualized Volatility (σ): {annualized_vol:.2%}")
print("--------------------------------------------------")
print(f"Current Stock Price: ${price:.2f}")
print("--------------------------------------------------")
print(f"Market Option Price (from data): ${opt_price:.2f}")
print(f"Calculated Option Price (Black-Scholes): ${price_option:.2f}")
print("--------------------------------------------------")
